<a href="https://colab.research.google.com/github/asepehri93/MLCC-LifeTime-Prediction/blob/main/src/Physics_based_MTTF_Prediction_MLCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import RepeatedKFold, cross_val_score

import xgboost as xgb
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import loguniform, uniform
from sklearn.ensemble import StackingRegressor


import seaborn as sns
sns.set_theme('poster')
sns.set_style('ticks')

random_seed = 42
np.random.seed(random_seed)

with open('Data.csv', 'r') as f:
    exp_df = pd.read_csv(f)

In [ ]:
def get_features(df_orig):
    df = df_orig.copy()
    df['V'] = df_orig['V']
    df['T'] = df_orig['T']
    df['V2'] = df['V'].apply(lambda x: x**2)
    df['V4'] = df['V'].apply(lambda x: x**4)
    df['V*T'] = df['V'] * df['T']
    df['V/T'] = df['V'] / df['T']
    df['LnV'] = df['V'].apply(lambda x: math.log(x))
    df['LnT'] = df['T'].apply(lambda x: math.log(x))
    df['TLnV'] = df['T'] * df['LnV']
    df['VLnT'] = df['V'] * df['LnT']
    return df

def get_TPM_MTTF(V, T, Beta, C):
    T += 273.15
    MTTF = np.exp(C-np.log(np.sinh(Beta*V/T)))/3600
    return MTTF

def generate_TPM_dataset(T, Beta, C):
    Vs, MTTFs = [], []
    for V in np.linspace(200, 400, 101):
        MTTF = get_TPM_MTTF(V, T, Beta, C)
        MTTFs.append(MTTF)
        Vs.append(V)
    df = pd.DataFrame()
    df['V'] = Vs
    df['T'] = [T] * len(Vs)
    df = get_features(df)
    df['MTTF'] = MTTFs
    return df

def train_base_model(data):
    params = {
        "learning_rate": [0.01, 0.1],
        "max_depth": [3, 4],
        "n_estimators": [100, 200]}
    
    model = xgb.XGBRegressor(objective='reg:squarederror')
    cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

    X = data.iloc[:,:-1]
    y = data.iloc[:, -1]
    grid_search = GridSearchCV(model, params, cv=cv, scoring="neg_mean_squared_error")
    grid_search.fit(X, y)
    best_params = grid_search.best_params_

    model = xgb.XGBRegressor(objective='reg:squarederror')
    model.set_params(**best_params)
    model.fit(X, y)
    return model, best_params

def train_transferred_model(data, model, best_params):
    transferred_model = xgb.XGBRegressor(objective='reg:squarederror')
    transferred_model.set_params(**best_params)
    X = data.iloc[:,:-1]
    y = data.iloc[:, -1]
    transferred_model.fit(X, y, xgb_model = model)
    return transferred_model


In [ ]:
T_list = [135, 140, 150, 165]
df = get_features(exp_df)
df = df[['V', 'T', 'V2', 'V4', 'V*T', 'V/T', 'LnV', 'LnT', 'TLnV', 'VLnT', 'MTTF', 'n', 'Beta', 'C']]
results_dict = {}

for T in T_list:
    exp_results, ml_results, tpm_results, Vs, Ts = [], [], [], [], []
    df_base = df[df['T'] == T].reset_index(drop=True)
    for index, row in df_base.iterrows():
        Ts.append(T)
        V0 = row.iloc[0]
        Vs.append(V0)
        Beta = row.iloc[-2]
        C = row.iloc[-1]
        yhat_exp = row.iloc[-4]
        df_TPM = generate_TPM_dataset(T, Beta, C)
        base_model, best_params = train_base_model(df_TPM)
        df_transfer = df_base.drop(index = index).reset_index(drop=True).iloc[:, :-3]
        transferred_model = train_transferred_model(df_transfer, base_model, best_params)
        yhat_ml = transferred_model.predict(pd.DataFrame(row.iloc[:-4]).T)
        yhat_tpm = get_TPM_MTTF(V0, T, Beta, C)
        exp_results.append(yhat_exp)
        ml_results.append(yhat_ml[0])
        tpm_results.append(yhat_tpm)
    results_dict[str(T)] = pd.DataFrame({'V': Vs, 'T': Ts, 'MLM':ml_results, 'TPM':tpm_results, 'Experiment': exp_results})

for T in T_list:
    df_base = df[df['T'] == T].reset_index(drop=True)
    em_results = []
    for index, row in df_base.iterrows():
        n = row.iloc[-3]
        rem_df = df_base.drop(index = index).reset_index(drop=True)
        V0 = row.iloc[0]
        random_row = rem_df.sample(n=1)
        V1 = random_row.iloc[:,0]
        mttf1 = random_row.iloc[:, -4]
        mttf0 = mttf1/((V0/V1) ** n)
        em_results.append(mttf0)
    results_dict[str(T)]['EM'] = em_results


for T in T_list:
    plot_df = results_dict[str(T)]
    Vs = plot_df['V']
    tpm_results = plot_df['TPM']
    ml_results = plot_df['MLM']
    em_results = plot_df['EM']
    true_results = plot_df['Experiment']
    mse_tpm = mean_squared_error(true_results, tpm_results)
    mse_ml = mean_squared_error(true_results, ml_results)
    mse_em = mean_squared_error(true_results, em_results)
    fig, ax = plt.subplots(figsize=(10,10), dpi=300)
    ax.plot(Vs, tpm_results, label = 'TPM', linestyle=':', marker='o')
    ax.plot(Vs, ml_results, label = 'MLM', linestyle=':', marker='s')
    ax.plot(Vs, em_results, label = 'EM', linestyle=':', marker='P')
    ax.plot(Vs, true_results, label='Experiment', linestyle=':', marker='^')
    ax.set_xlabel('V (v)')
    ax.set_ylabel('MTTF (hr)')
    ax.set_title('Model performance for $T = {}^\circ$C'.format(T))
    ax.legend(frameon=False)
    ax.text(0.05, 0.25, f'$mse_{{TPM}}$: {round(mse_tpm, 2)}\n$mse_{{MLM}}$: {round(mse_ml, 2)}\n$mse_{{EM}}$: {round(mse_em, 2)}',
             ha='left', va='top',transform=plt.gca().transAxes)
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax = plt.gca()
    yticks = ax.get_yticks()

    if 0 not in yticks:
        yticks = np.append(0, yticks)
    ax.minorticks_off()
    ax.tick_params(axis='both', which='major', direction='in', length=10, width=2)
